In [ ]:
import semopy
print("semopy version:", semopy.__version__)


semopy version: 2.3.11


In [ ]:
!pip install --upgrade semopy


In [ ]:
# Install necessary libraries
!pip install pandas openpyxl

# Import Libraries
import pandas as pd
import numpy as np

# Set random seed for reproducibility
np.random.seed(42)

# Number of samples
n_samples = 300

# Generate latent variables
PQ = np.random.normal(0, 1, n_samples)  # Product Quality
SQ = np.random.normal(0, 1, n_samples)  # Service Quality
VM = np.random.normal(0, 1, n_samples)  # Value for Money

# Generate observed variables with specified loadings and measurement error
PQ1 = 0.8 * PQ + np.random.normal(0, 0.6, n_samples)
PQ2 = 0.85 * PQ + np.random.normal(0, 0.6, n_samples)
PQ3 = 0.9 * PQ + np.random.normal(0, 0.6, n_samples)

SQ1 = 0.75 * SQ + np.random.normal(0, 0.6, n_samples)
SQ2 = 0.8 * SQ + np.random.normal(0, 0.6, n_samples)
SQ3 = 0.85 * SQ + np.random.normal(0, 0.6, n_samples)

VM1 = 0.8 * VM + np.random.normal(0, 0.6, n_samples)
VM2 = 0.85 * VM + np.random.normal(0, 0.6, n_samples)
VM3 = 0.9 * VM + np.random.normal(0, 0.6, n_samples)

# Combine into a DataFrame
data = pd.DataFrame({
    'PQ1': PQ1,
    'PQ2': PQ2,
    'PQ3': PQ3,
    'SQ1': SQ1,
    'SQ2': SQ2,
    'SQ3': SQ3,
    'VM1': VM1,
    'VM2': VM2,
    'VM3': VM3
})

# Transform to Likert scale (1 to 7)
for col in data.columns:
    # Standardize the data
    data[col] = (data[col] - data[col].min()) / (data[col].max() - data[col].min())
    # Scale to 1-7 Likert scale
    data[col] = data[col] * 6 + 1
    # Round to simulate survey responses
    data[col] = data[col].round()

# Ensure values are within 1 to 7
data = data.clip(lower=1, upper=7)

# Display the first few rows
print(data.head())

# Save data to Excel file
data.to_excel('customer_satisfaction.xlsx', index=False)

# If using Google Colab, download the file
try:
    from google.colab import files
    files.download('customer_satisfaction.xlsx')
except ImportError:
    pass  # Not running in Google Colab


   PQ1  PQ2  PQ3  SQ1  SQ2  SQ3  VM1  VM2  VM3
0  4.0  4.0  5.0  5.0  4.0  3.0  4.0  3.0  4.0
1  3.0  3.0  4.0  4.0  3.0  4.0  4.0  3.0  2.0
2  4.0  4.0  4.0  5.0  4.0  4.0  6.0  4.0  4.0
3  5.0  5.0  5.0  4.0  5.0  4.0  5.0  6.0  4.0
4  4.0  4.0  4.0  3.0  3.0  5.0  4.0  5.0  5.0


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Install necessary libraries
!pip install --upgrade semopy pandas openpyxl

# Import Libraries
import pandas as pd
import numpy as np
from semopy import Model
import matplotlib.pyplot as plt
import seaborn as sns

# Check semopy version
print("semopy version:", semopy.__version__)

# Load data from Excel file
data = pd.read_excel('customer_satisfaction.xlsx')

# Display the first few rows
print(data.head())

# Step 1: Specify the CFA Model

model_desc = """
# Latent Variables
PQ =~ PQ1 + PQ2 + PQ3
SQ =~ SQ1 + SQ2 + SQ3
VM =~ VM1 + VM2 + VM3

# Covariances between latent variables
PQ ~~ SQ
PQ ~~ VM
SQ ~~ VM
"""

# Step 2: Create and Fit the Model

# Create the SEM model
model = Model(model_desc)

# Fit the model
res = model.fit(data)

# Get the results and inspect the structure of the DataFrame
estimates = model.inspect(std_est=True)  # Include standardized estimates
print("\nColumn Names in estimates DataFrame:\n", estimates.columns)  # Print column names
print("\nFirst few rows of estimates DataFrame:\n", estimates.head())   # Print the first few rows to inspect the data structure

# Step 3: Evaluate Model Fit

# semopy provides some basic model fit statistics in the fit summary.
print("\nModel Fit Summary:")
print(res)

# Step 4: Extract Parameter Estimates

print("\nParameter Estimates:")
print(estimates)

# Step 5: Reliability Analysis

def cronbach_alpha(items):
    items = np.asarray(items)
    item_variances = items.var(axis=1, ddof=1)
    total_score = items.sum(axis=0)
    n_items = items.shape[0]
    return (n_items / (n_items - 1)) * (1 - (item_variances.sum() / total_score.var(ddof=1)))

# Extract items for each construct
PQ_items = data[['PQ1', 'PQ2', 'PQ3']].values.T
SQ_items = data[['SQ1', 'SQ2', 'SQ3']].values.T
VM_items = data[['VM1', 'VM2', 'VM3']].values.T

# Calculate Cronbach's alpha
alpha_PQ = cronbach_alpha(PQ_items)
alpha_SQ = cronbach_alpha(SQ_items)
alpha_VM = cronbach_alpha(VM_items)

print(f"\nCronbach's alpha for Product Quality (PQ): {alpha_PQ:.2f}")
print(f"Cronbach's alpha for Service Quality (SQ): {alpha_SQ:.2f}")
print(f"Cronbach's alpha for Value for Money (VM): {alpha_VM:.2f}")

# Step 6: Validity Assessment

# Convergent Validity - Calculate AVE

# List of latent constructs
constructs = ['PQ', 'SQ', 'VM']

# Filter loadings from estimates: find rows where 'op' is '~' (this means it's a loading)
loadings = estimates[(estimates['op'] == '~') & (estimates['rval'].isin(constructs))][['lval', 'rval', 'Est. Std']]

# Map loadings to constructs
PQ_loadings = loadings[loadings['rval'] == 'PQ']['Est. Std'].values
SQ_loadings = loadings[loadings['rval'] == 'SQ']['Est. Std'].values
VM_loadings = loadings[loadings['rval'] == 'VM']['Est. Std'].values

# Calculate AVE
def calculate_ave(loadings):
    return np.mean(loadings ** 2)

AVE_PQ = calculate_ave(PQ_loadings)
AVE_SQ = calculate_ave(SQ_loadings)
AVE_VM = calculate_ave(VM_loadings)

print(f"\nAVE for Product Quality (PQ): {AVE_PQ:.2f}")
print(f"AVE for Service Quality (SQ): {AVE_SQ:.2f}")
print(f"AVE for Value for Money (VM): {AVE_VM:.2f}")

# Discriminant Validity - Fornell-Larcker Criterion

# Calculate the square root of AVE
sqrt_AVE_PQ = np.sqrt(AVE_PQ)
sqrt_AVE_SQ = np.sqrt(AVE_SQ)
sqrt_AVE_VM = np.sqrt(AVE_VM)

# Compute correlations between constructs
latent_vars = model.predict_factors(data)
correlations = np.corrcoef(latent_vars.T)

print("\nCorrelations between latent constructs:")
constructs = ['PQ', 'SQ', 'VM']
correlation_df = pd.DataFrame(correlations, index=constructs, columns=constructs)
print(correlation_df)

print(f"\nSquare root of AVE for PQ: {sqrt_AVE_PQ:.2f}")
print(f"Square root of AVE for SQ: {sqrt_AVE_SQ:.2f}")
print(f"Square root of AVE for VM: {sqrt_AVE_VM:.2f}")

# Compare square roots of AVE with inter-construct correlations
print("\nDiscriminant Validity Check (Fornell-Larcker Criterion):")
sqrt_AVEs = [sqrt_AVE_PQ, sqrt_AVE_SQ, sqrt_AVE_VM]
for i in range(3):
    for j in range(3):
        if i != j:
            print(f"Square root of AVE for {constructs[i]} vs. Correlation({constructs[i]},{constructs[j]}): {sqrt_AVEs[i]:.2f} vs. {correlations[i,j]:.2f}")


semopy version: 2.3.11
   PQ1  PQ2  PQ3  SQ1  SQ2  SQ3  VM1  VM2  VM3
0    4    4    5    5    4    3    4    3    4
1    3    3    4    4    3    4    4    3    2
2    4    4    4    5    4    4    6    4    4
3    5    5    5    4    5    4    5    6    4
4    4    4    4    3    3    5    4    5    5

Column Names in estimates DataFrame:
 Index(['lval', 'op', 'rval', 'Estimate', 'Est. Std', 'Std. Err', 'z-value',
       'p-value'],
      dtype='object')

First few rows of estimates DataFrame:
   lval op rval  Estimate  Est. Std  Std. Err    z-value p-value
0  PQ1  ~   PQ  1.000000  0.778878         -          -       -
1  PQ2  ~   PQ  1.214318  0.752678  0.099577  12.194728     0.0
2  PQ3  ~   PQ  1.563551  0.836808  0.124327  12.576071     0.0
3  SQ1  ~   SQ  1.000000  0.754750         -          -       -
4  SQ2  ~   SQ  0.998154  0.734240    0.0881  11.329835     0.0

Model Fit Summary:
Name of objective: MLW
Optimization method: SLSQP
Optimization successful.
Optimization termin